In [1]:
import urllib.request

f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

In [2]:
from pyspark import SparkContext

sc=SparkContext('local[*]')
data_file = "kddcup.data.gz"
raw_data = sc.textFile(data_file)

print("Training data size is {}".format(raw_data.count()))

Training data size is 4898431


In [3]:
raw_data.take(5)

['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,162,4528,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1,1,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,236,1228,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,2,2,1.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,3,3,1.00,0.00,0.33,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,3,0.00,0.00,0.00,0.00,1.00,0.00,0.00,4,4,1.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,normal.']

In [4]:
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")

test_data_file = "corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print("Testing data size is {}".format(test_raw_data.count()))

Testing data size is 311029


In [5]:
test_raw_data.take(5)

['0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,snmpgetattack.',
 '0,udp,private,SF,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,254,1.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,snmpgetattack.']

In [6]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

csv_data = raw_data.map(lambda x: x.split(","))
test_csv_data = test_raw_data.map(lambda x: x.split(","))

In [7]:
print("*** raw_data RDD ***")
print(raw_data.take(1))
print("\n*** csv_data RDD ***")
print(csv_data.take(1))

*** raw_data RDD ***
['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.']

*** csv_data RDD ***
[['0', 'tcp', 'http', 'SF', '215', '45076', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '0', '0', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]


In [8]:
protocols = csv_data.map(lambda x: x[1]).distinct().collect()
services = csv_data.map(lambda x: x[2]).distinct().collect()
flags = csv_data.map(lambda x: x[3]).distinct().collect()

In [9]:
# 看services中的元素為何
print(services)
# 看看telnet是services中的第幾個元素
print(services.index('telnet'))

['http', 'smtp', 'domain_u', 'auth', 'finger', 'telnet', 'eco_i', 'ftp', 'ntp_u', 'ecr_i', 'other', 'urp_i', 'private', 'pop_3', 'ftp_data', 'netstat', 'daytime', 'ssh', 'echo', 'time', 'name', 'whois', 'domain', 'mtp', 'gopher', 'remote_job', 'rje', 'ctf', 'supdup', 'link', 'systat', 'discard', 'X11', 'shell', 'login', 'imap4', 'nntp', 'uucp', 'pm_dump', 'IRC', 'Z39_50', 'netbios_dgm', 'ldap', 'sunrpc', 'courier', 'exec', 'bgp', 'csnet_ns', 'http_443', 'klogin', 'printer', 'netbios_ssn', 'pop_2', 'nnsp', 'efs', 'hostnames', 'uucp_path', 'sql_net', 'vmnet', 'iso_tsap', 'netbios_ns', 'kshell', 'urh_i', 'http_2784', 'harvest', 'aol', 'tftp_u', 'http_8001', 'tim_i', 'red_i']
5


In [18]:
def create_labeled_point(line_split):
    # leave_out = [41]
    clean_line_split = line_split[0:41]
    #print(clean_line_split)

    
    # convert protocol to numeric categorical variable
    try: 
        clean_line_split[1] = protocols.index(clean_line_split[1])
    except:
        clean_line_split[1] = len(protocols)
        
    # convert service to numeric categorical variable
    try:
        clean_line_split[2] = services.index(clean_line_split[2])
    except:
        clean_line_split[2] = len(services)
    
    # convert flag to numeric categorical variable
    try:
        clean_line_split[3] = flags.index(clean_line_split[3])
    except:
        clean_line_split[3] = len(flags)
    
    # convert label to binary label
    attack = 1.0
    if "normal" in line_split[41] :
        attack = 0.0
        
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [21]:
training_data = csv_data.map(create_labeled_point)
test_data = test_csv_data.map(create_labeled_point)

In [22]:
print("*** Training data ***")
print(training_data.take(1))
print("*** Testing data ***")
print(test_data.take(1))

*** Training data ***
[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,215.0,45076.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])]
*** Testing data ***
[LabeledPoint(0.0, [0.0,1.0,12.0,0.0,105.0,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,255.0,254.0,1.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0])]


In [23]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from time import time

# 請依照要求設定decision tree參數：
# 1. numClasses = ?
# 2. categoricalFeaturesInfo=?
# 3. 利用gini index來當作計算impurity的標準
# 4. Terminal Condition: 最深展開到4層，資料小於100個則停止
# 5. maxBins = 100
t0 = time()
tree_model = DecisionTree.trainClassifier(training_data, numClasses=2, categoricalFeaturesInfo={},
            impurity='gini', maxDepth=4, maxBins=100)
tt = time() - t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 579.303 seconds


In [26]:
predictions = tree_model.predict(test_data.map(lambda x: x.features))
labels_and_preds = test_data.map(lambda lp: lp.label).zip(predictions)

In [27]:
labels_and_preds.take(1)

[(0.0, 0.0)]

In [28]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda lp: lp[0]==lp[1]).count() / test_data.count() *1.0
tt = time() - t0

print("Prediction made in {} seconds. Test accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

Prediction made in 89.904 seconds. Test accuracy is 0.9226


In [29]:
# 印出decision tree
print("Learned classification tree model:")
print(tree_model.toDebugString())

Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 29 nodes
  If (feature 22 <= 33.0)
   If (feature 3 <= 6.0)
    If (feature 36 <= 0.48)
     If (feature 12 <= 0.0)
      Predict: 0.0
     Else (feature 12 > 0.0)
      Predict: 1.0
    Else (feature 36 > 0.48)
     If (feature 2 <= 5.0)
      Predict: 0.0
     Else (feature 2 > 5.0)
      Predict: 1.0
   Else (feature 3 > 6.0)
    If (feature 33 <= 0.29)
     If (feature 5 <= 0.0)
      Predict: 1.0
     Else (feature 5 > 0.0)
      Predict: 0.0
    Else (feature 33 > 0.29)
     If (feature 22 <= 2.0)
      Predict: 0.0
     Else (feature 22 > 2.0)
      Predict: 1.0
  Else (feature 22 > 33.0)
   If (feature 5 <= 0.0)
    If (feature 11 <= 0.0)
     If (feature 36 <= 0.09)
      Predict: 1.0
     Else (feature 36 > 0.09)
      Predict: 1.0
    Else (feature 11 > 0.0)
     If (feature 4 <= 6.0)
      Predict: 1.0
     Else (feature 4 > 6.0)
      Predict: 0.0
   Else (feature 5 > 0.0)
    If (feature 29 <=